In [1]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

from itertools import combinations
from itertools import permutations

In [2]:
demand = pd.read_excel("demand.xlsx")

In [3]:
demand

,shop,January,February,March,April,May,June,July,August,September,October,November,December
0,1,4.4,4.0,4.8,6.8,6.4,6.4,6.4,6.8,5.6,5.6,3.6,2.8
1,2,4.0,4.0,4.8,6.0,7.2,4.8,6.4,6.8,4.8,5.6,4.0,4.0
2,3,5.2,6.0,6.8,8.8,8.0,8.0,8.8,8.0,7.6,7.2,5.6,4.8
3,4,6.0,5.2,7.6,7.2,8.8,8.8,7.2,8.0,6.8,6.4,4.8,4.4
4,5,4.0,4.8,6.8,5.2,7.2,5.6,6.4,7.6,4.8,5.2,4.0,3.2
5,6,6.0,6.4,6.0,7.6,7.2,8.8,11.2,9.2,7.6,6.8,5.6,4.8
6,7,9.2,8.0,11.6,11.2,10.8,10.4,12.4,11.2,11.2,11.2,7.2,6.0
7,8,7.2,7.6,10.0,12.8,18.8,12.8,13.6,12.4,12.0,10.0,8.0,6.8
8,9,10.4,9.2,13.6,14.8,16.4,16.0,16.0,11.6,13.2,10.4,8.0,6.4
9,10,7.6,7.6,7.2,9.6,12.8,9.6,11.2,9.6,10.4,8.8,6.0,6.0


In [4]:
shops = demand.index #all the shop names


In [5]:
def rSubset(shops, r):
  
    # return list of all subsets of length r
    # to deal with duplicate subsets use 
    # set(list(combinations(shops, r)))
    return list(combinations(shops, r))

In [6]:
list_of_sets_array = pd.DataFrame()
  
for r in range(0, 5): #the max amount of shops per set is 4
    for i in range(0,len(rSubset(shops, r))):
        vol_sum = 0
        for j in range(0,r):
            vol_sum = vol_sum + demand['January'][rSubset(shops, r)[i][j]] #value of shop 
        
        if vol_sum <= 20: #the max value of one truck is 20 m3
            temp = pd.DataFrame(
                {rSubset(shops, r)[i]}
            )
            list_of_sets_array = pd.concat([list_of_sets_array, temp]) 
            
list_of_sets_array = list_of_sets_array.iloc[1: , :].reset_index().drop(['index'], axis=1)


In [7]:
list_of_sets_array #we could add a column of the best fitting warehouse for each set to this dataframe

,0,1,2,3
0,0.0,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN
2,2.0,NaN,NaN,NaN
3,3.0,NaN,NaN,NaN
4,4.0,NaN,NaN,NaN
...,...,...,...,...
675,3.0,4.0,17.0,18.0
676,3.0,4.0,18.0,19.0
677,4.0,5.0,17.0,18.0
678,4.0,5.0,18.0,19.0


In [8]:
db = pd.DataFrame([0]) #creating a dataframe with the right structure
list_of_sets_per_shop = pd.DataFrame() #creating a dataframe to store the lists into

for n in range(0,20):

    for i in range(0,len(list_of_sets_array)):
        db[i] = shops[n] in list(list_of_sets_array.iloc[i])
        #db[i] = pd.DataFrame([list_of_sets_string['subset'][i].find(shops[n])]) #this still doesn't work, it sees 11, 12 etc as 1 and 20 as 2
    
    
    tr = db.transpose().rename(columns={0:shops[n]})
    tr[tr[shops[n]] == True] = 1
    tr[tr[shops[n]] == False] = 0
    
    #contains = pd.DataFrame(list_of_sets[tr]['subset'].dropna())
    
    list_of_sets_per_shop = pd.concat([list_of_sets_per_shop, tr[shops[n]]], axis=1, join="outer")

In [9]:
list_of_sets_per_shop.transpose()

,0,1,2,3,4,5,6,7,8,9,...,670,671,672,673,674,675,676,677,678,679
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,1,1,1,1,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,1
5,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
6,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [10]:
array_of_sets = []
for i in range(1,len(list_of_sets_array)):
    array_of_sets += [list(list_of_sets_array.iloc[i])]

In [22]:
sets = list_of_sets_per_shop.index

# Distance of each set

In [12]:
# Matching score data this is basically creating the matrix hier moet een lijst staan met alle sets en hun distances
multidict_input = {} #is this multidict the matrix of list_per_sets_per_shop and the row vector of the distances?

for i in range(len(array_of_sets)):     
    multidict_input[tuple(array_of_sets[i])] = 5

keys, distances = gp.multidict(multidict_input)  # feed it into multidict

In [13]:
distances

{(1.0, nan, nan, nan): 5,
 (2.0, nan, nan, nan): 5,
 (3.0, nan, nan, nan): 5,
 (4.0, nan, nan, nan): 5,
 (5.0, nan, nan, nan): 5,
 (6.0, nan, nan, nan): 5,
 (7.0, nan, nan, nan): 5,
 (8.0, nan, nan, nan): 5,
 (9.0, nan, nan, nan): 5,
 (10.0, nan, nan, nan): 5,
 (11.0, nan, nan, nan): 5,
 (12.0, nan, nan, nan): 5,
 (13.0, nan, nan, nan): 5,
 (14.0, nan, nan, nan): 5,
 (15.0, nan, nan, nan): 5,
 (16.0, nan, nan, nan): 5,
 (17.0, nan, nan, nan): 5,
 (18.0, nan, nan, nan): 5,
 (19.0, nan, nan, nan): 5,
 (0.0, 1.0, nan, nan): 5,
 (0.0, 2.0, nan, nan): 5,
 (0.0, 3.0, nan, nan): 5,
 (0.0, 4.0, nan, nan): 5,
 (0.0, 5.0, nan, nan): 5,
 (0.0, 6.0, nan, nan): 5,
 (0.0, 7.0, nan, nan): 5,
 (0.0, 8.0, nan, nan): 5,
 (0.0, 9.0, nan, nan): 5,
 (0.0, 10.0, nan, nan): 5,
 (0.0, 11.0, nan, nan): 5,
 (0.0, 12.0, nan, nan): 5,
 (0.0, 13.0, nan, nan): 5,
 (0.0, 14.0, nan, nan): 5,
 (0.0, 15.0, nan, nan): 5,
 (0.0, 16.0, nan, nan): 5,
 (0.0, 17.0, nan, nan): 5,
 (0.0, 18.0, nan, nan): 5,
 (0.0, 19.0, nan, n

In [14]:
# Declare and initialize model
m = gp.Model('RAP')

Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "TUELICENSE4"


In [15]:
# Create decision variables for the RAP model
x = m.addVars(keys, name="assign")

In [16]:
# Create set constraints
set_const = m.addConstrs((x.sum('*',st) <= 1 for st in sets), name='set') #preferably 0 or 1 (<= 1) as not all sets end up in the final selection

In [17]:
# Create shops constraints
shop_const = m.addConstrs((x.sum(shop,'*') == 1 for shop in shops), name='shop') #but all shops have to be included so ( == 1), but shops are in sets so how do we do that, I need to implement the matrix of shops in sets

In [18]:
# Objective: maximize total matching score of all assignments
m.setObjective(x.prod(distances), GRB.MINIMIZE)

In [19]:
# Save model for inspection
m.write('RAP.lp')

In [20]:
# Run optimization engine
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 700 rows, 679 columns and 1339 nonzeros
Model fingerprint: 0x289e0933
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 664 rows and 479 columns
Presolve time: 0.01s
Presolved: 36 rows, 200 columns, 382 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+02   0.000000e+00   0.000000e+00      0s
       0    1.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.000000000e+02


In [21]:
# Display optimal values of decision variables
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total matching score: ', m.objVal)

assign[1.0,nan,nan,nan] 1.0
assign[2.0,nan,nan,nan] 1.0
assign[3.0,nan,nan,nan] 1.0
assign[4.0,nan,nan,nan] 1.0
assign[5.0,nan,nan,nan] 1.0
assign[6.0,nan,nan,nan] 1.0
assign[7.0,nan,nan,nan] 1.0
assign[8.0,nan,nan,nan] 1.0
assign[9.0,nan,nan,nan] 1.0
assign[10.0,nan,nan,nan] 1.0
assign[11.0,nan,nan,nan] 1.0
assign[12.0,nan,nan,nan] 1.0
assign[13.0,nan,nan,nan] 1.0
assign[14.0,nan,nan,nan] 1.0
assign[15.0,nan,nan,nan] 1.0
assign[16.0,nan,nan,nan] 1.0
assign[17.0,nan,nan,nan] 1.0
assign[18.0,nan,nan,nan] 1.0
assign[19.0,nan,nan,nan] 1.0
assign[0.0,1.0,9.0,nan] 1.0
Total matching score:  100.0
